In [1]:
from mitools.visuals.plots import PlotComposer, ScatterPlotter, LinePlotter, AxesComposer
import pandas as pd
import numpy as np
from mitools.pandas import idxslice
import matplotlib.pyplot as plt

# Plots and Axes Composers Examples

This notebook contains examples of how to use the `PlotComposer` and `AxesComposer` classes to create plots and axes composers, portraying their functionality and flexibility.

## Artifical Data

#### Functions to Generate Countries Indicators

In [2]:
def generate_gdp(year_idx: int, base_gdp: float, growth_rate: float) -> float:
    gdp = base_gdp * ((1.0 + growth_rate) ** year_idx)
    noise = np.random.normal(loc=0.0, scale=0.05)  # ±5% relative noise
    return gdp * (1 + noise)

def generate_population(year_idx: int, start_pop: float, end_pop: float, total_years: int) -> float:
    fraction = year_idx / (total_years - 1)
    pop = start_pop + (end_pop - start_pop) * fraction
    noise_factor = 0.03 * pop
    pop += np.random.normal(loc=0.0, scale=noise_factor)
    return max(pop, 0)  # population can't be negative

def generate_unemployment(year_idx: int) -> float:
    base = 0.1 + 0.1 * np.sin(2 * np.pi * (year_idx / 10.0))  # 10-year cycles
    noise = np.random.normal(loc=0.0, scale=0.03)
    val = np.clip(base + noise, 0, 0.25)
    return val * 100

def generate_happiness(year_idx: int, previous_val: float) -> float:
    step = np.random.normal(loc=0.0, scale=0.3)
    new_val = previous_val + step
    new_val = np.clip(new_val, 0, 10)
    return new_val

def generate_co2(year_idx: int, base_co2: float, growth_rate: float) -> float:
    co2_val = base_co2 * ((1.0 + growth_rate) ** year_idx)
    noise = np.random.normal(loc=0.0, scale=0.05)
    return co2_val * (1 + noise)

#### Initialize Country Data

In [ ]:
np.random.seed(42)
years = range(1900, 2021)
n_years = len(years)
n_indicators = 5
continents = {
    "Africa":    ["Algeria", "Nigeria", "Kenya", "Egypt", "South Africa"],
    "Asia":      ["China", "India", "Japan", "South Korea", "Indonesia"],
    "Europe":    ["Germany", "France", "Italy", "Spain", "Poland"],
    "Americas":  ["USA", "Canada", "Brazil", "Argentina", "Mexico"],
    "Oceania":   ["Australia", "New Zealand", "Fiji", "Papua New Guinea", "Samoa"]
}
country_continent_pairs = []
for cont, countries in continents.items():
    for c in countries:
        country_continent_pairs.append((c, cont))
country_params = {}
for c, cont in country_continent_pairs:
    base_gdp = np.random.uniform(1e3, 1e4)
    gdp_growth = np.random.uniform(0.02, 0.08)
    start_pop = np.random.uniform(1e5, 5e6)
    end_pop = np.random.uniform(5e6, 2e8)
    base_co2 = np.random.uniform(1, 500)
    co2_growth = np.random.uniform(0.01, 0.05)
    country_params[c] = {
        "continent": cont,
        "base_gdp": base_gdp,
        "gdp_growth": gdp_growth,
        "start_pop": start_pop,
        "end_pop": end_pop,
        "base_co2": base_co2,
        "co2_growth": co2_growth
    }
all_rows = []
for c, cont in country_continent_pairs:
    params = country_params[c]
    current_happiness = np.random.uniform(0, 10)
    for i, y in enumerate(years):
        gdp_val = generate_gdp(
            year_idx=i, 
            base_gdp=params["base_gdp"], 
            growth_rate=params["gdp_growth"]
        )
        pop_val = generate_population(
            year_idx=i,
            start_pop=params["start_pop"],
            end_pop=params["end_pop"],
            total_years=n_years
        )
        unemp_val = generate_unemployment(i)
        current_happiness = generate_happiness(i, current_happiness)
        co2_val = generate_co2(
            year_idx=i,
            base_co2=params["base_co2"],
            growth_rate=params["co2_growth"]
        )
        row = {
            "Country": c,
            "Continent": cont,
            "Year": y,
            "GDP": gdp_val,
            "Population": pop_val,
            "UnemploymentRate": unemp_val,
            "HappinessIndex": current_happiness,
            "CO2": co2_val
        }
        all_rows.append(row)
countries_data = pd.DataFrame(all_rows).set_index(["Year", "Continent", "Country"])
countries_data

## General Definitions

In [4]:
continents = countries_data.index.get_level_values("Continent").unique()
continents_col = "Continent"
years = countries_data.index.get_level_values("Year").unique()
indicators = [c for c in countries_data.columns if c != "GDP"]

In [5]:
y_var = "GDP"
x_var = "Population"

In [6]:
edgecolors = {
    "Population": "grey",
    "GDP": "green",
    "UnemploymentRate": "orange",
    "HappinessIndex": "brown",
    "CO2": "red",
}
facecolors = {
    "Africa": "white",
    "Asia": "darkgrey",
    "Europe": "lightgrey",
    "Americas": "black",
    "Oceania": "lightcyan",
}

## PlotComposer with multiple ScatterPlotters

In [ ]:
plot = PlotComposer()
for continent in continents:
    continent_idxslice = idxslice(countries_data, level=continents_col, values=continent, axis=0)
    plot.add_plotter(
        ScatterPlotter(
            x_data=countries_data.loc[continent_idxslice, x_var].values,
            y_data=np.log(countries_data.loc[continent_idxslice, y_var].values),
        )
        .set_edgecolor(edgecolors[x_var])
        .set_facecolor(facecolors[continent])
        .set_marker("o")
        .set_size(20)
        .set_alpha(0.75)
    )
plot.set_title(f"a) {y_var} vs {x_var}:", fontsize=24, fontweight="bold", loc="left", fontfamily="serif")
plot.set_xlabel(f"{x_var}", fontsize=16, fontfamily="serif")
plot.set_ylabel(f"{y_var}", fontsize=16, fontfamily="serif")
ax = plot.draw()

## Store Created PlotComposer

In [8]:
plot.save_composer(".plot_params/composed_plot.json")

## Load and Draw Created PlotComposer

In [ ]:
plot = PlotComposer.from_json(".plot_params/composed_plot.json")
ax = plot.draw()

## Generate Multiple PlotComposers Independently

In [ ]:
plots = []
for n, var in enumerate(indicators):
    plot = PlotComposer()
    for continent in continents:
        continent_idxslice = idxslice(countries_data, level=continents_col, values=continent, axis=0)
        plot.add_plotter(
            ScatterPlotter(
                x_data=countries_data.loc[continent_idxslice, var].values,
                y_data=np.log(countries_data.loc[continent_idxslice, y_var].values),
            )
            .set_edgecolor(edgecolors[var])
            .set_facecolor(facecolors[continent])
            .set_marker("o")
            .set_size(20)
            .set_alpha(0.75)
        )
    plot.set_title(f"{'abcd'[n]}) {y_var} vs {var}:", fontsize=24, fontweight="bold", loc="left", fontfamily="serif")
    plot.set_xlabel(f"{var}", fontsize=16, fontfamily="serif")
    plot.set_ylabel(f"{y_var}", fontsize=16, fontfamily="serif")
    plots.append(plot)
    plot.save_composer(".plot_params/composed_plot.json")
    plot = PlotComposer.from_json(".plot_params/composed_plot.json")
    plot.draw()

## Relate Multiple PlotComposers with an AxesComposer

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(8*2,8*2))
plotted = AxesComposer(axes, plots=plots)
ax = plotted.draw()

### Store, Load and Draw the Created AxesComposer

In [ ]:
plotted.save_composer(".plot_params/composed_plot.json")
plotted = AxesComposer.from_json(".plot_params/composed_plot.json")
axes = plotted.draw()

## Individual PlotComposers in AxesComposer can be stored, loaded, and drawn independently

In [ ]:
for plot in plotted.plots:
    plot.save_composer(".plot_params/composed_plot.json")
    plot = PlotComposer.from_json(".plot_params/composed_plot.json").set_figsize((11,8))
    plot.draw()

***